# 📚 Content-Based Recommendation System Using Embeddings

## Overview
This notebook implements a **content-based recommender system** using OpenAI embeddings and cosine similarity for nearest neighbor search. It's designed for Google Colab environments and provides both programmatic and interactive (Gradio) interfaces.

### Key Features
- **Semantic Search**: Leverages OpenAI's `text-embedding-3-small` model for high-quality embeddings
- **Efficient Caching**: Stores embeddings to minimize API calls
- **Interactive UI**: Gradio interface for easy exploration
- **Scalable**: Works with large datasets using NumPy and scikit-learn

### Workflow
1. Install dependencies
2. Configure OpenAI API credentials
3. Load and prepare dataset
4. Generate embeddings with caching
5. Build recommendation engine
6. Launch interactive interface

In [2]:
%pip install --upgrade openai pandas numpy scikit-learn tqdm gradio

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached openai-2.15.0-py3-none-any.whl.metadata (29 kB)
  Using cached numpy-2.4.1-cp314-cp314-win_amd64.whl.metadata (6.6 kB)
  Using cached scikit_learn-1.8.0-cp314-cp314-win_amd64.whl.metadata (11 kB)
  Using cached gradio-6.3.0-py3-none-any.whl.metadata (16 kB)
  Using cached scipy-1.17.0-cp314-cp314-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached gradio_client-2.0.3-py3-none-any.whl.metadata (7.1 kB)
Using cached openai-2.15.0-py3-none-any.whl (1.1 MB)
Using cached numpy-2.4.1-cp314-cp314-win_amd64.whl (12.4 MB)
Using cached scikit_learn-1.8.0-cp314-cp314-win_amd64.whl (8.1 MB)
   ---------------------------------------- 0.0/23.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/23.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/23.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/23.0 MB ? eta -

ERROR: Could not install packages due to an OSError: [WinError 145] The directory is not empty: 'c:\\users\\aarru\\appdata\\local\\programs\\python\\python314\\lib\\site-packages\\openai\\types\\realtime\\__pycache__'



  Using cached openai-2.15.0-py3-none-any.whl.metadata (29 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   ------------------- -------------------- 0.5/1.1 MB 752.4 kB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 881.0 kB/s eta 0:00:01
   ---------------------------------------  1.0/1.1 MB 920.3 kB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 887.2 kB/s  0:00:01
   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.4 MB 1.4 MB/s eta 0:00:09
   -- ------------------------------------- 0.8/12.4 MB 1.4 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/12.4 MB 1.

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\aarru\\AppData\\Local\\Programs\\Python\\Python314\\Lib\\site-packages\\openai-2.15.0.dist-info\\INSTALLERwk7m1q86.tmp'



## 🔑 Step 1: Configure OpenAI API Credentials

**Purpose**: Set up authentication for accessing OpenAI's embedding API

**Requirements**:
- Valid OpenAI API key (get from [https://platform.openai.com/api-keys](https://platform.openai.com/api-keys))
- API key should have embedding generation permissions

**Security Note**: 
- ⚠️ Replace `YOUR_OPENAI_API_KEY` with your actual API key
- Consider using environment variables or secrets managers in production
- Never commit API keys to version control

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

## 📦 Step 2: Import Libraries and Initialize Configuration

**Purpose**: Load required dependencies and configure embedding model parameters

**Library Breakdown**:
- `pandas`: Data manipulation and CSV handling
- `numpy`: Numerical computations and array operations
- `pickle`: Serialization for caching embeddings
- `tqdm`: Progress visualization for loops
- `sklearn.metrics.pairwise`: Cosine similarity computation
- `openai`: Official OpenAI Python client

**Configuration**:
- `EMBEDDING_MODEL`: Uses `text-embedding-3-small` (cost-effective, high-quality)
- `CACHE_PATH`: Local storage for embeddings to reduce API calls
- `client`: Authenticated OpenAI API client instance

In [ ]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

client = OpenAI()
EMBEDDING_MODEL = "text-embedding-3-small"
CACHE_PATH = "embedding_cache.pkl"

## 📥 Step 3: Load Dataset

**Purpose**: Import the news articles dataset for recommendation analysis

**Dataset Requirements**:
- File: `AG_news_samples.csv`
- Must contain a `description` column with article text
- Expected format: Comma-separated values with headers

**Data Operations**:
- Reads CSV into pandas DataFrame
- Displays first 5 rows for verification
- Prepares data for embedding generation

**Note**: Ensure `AG_news_samples.csv` is in the same directory as this notebook

In [ ]:
df = pd.read_csv("AG_news_samples.csv")
df.head()

## 💾 Step 4: Embedding Cache Management

**Purpose**: Load or initialize a cache to store embeddings and reduce API costs

**Benefits**:
- **Cost Optimization**: Avoids redundant API calls for identical text
- **Performance**: Local retrieval is significantly faster than API calls
- **Persistence**: Cache persists between notebook sessions

**Implementation**:
- Attempts to load existing cache from `embedding_cache.pkl`
- Creates new empty dictionary if cache doesn't exist
- Populated during embedding generation phase

**Cache Structure**: `{(text, model): embedding_vector}`

In [ ]:
try:
    with open(CACHE_PATH, "rb") as f:
        embedding_cache = pickle.load(f)
except FileNotFoundError:
    embedding_cache = {}

## 🧠 Step 5: Embedding Generation Function

**Purpose**: Convert text into vector embeddings using OpenAI's API with caching support

**Function**: `get_embedding(text, model=EMBEDDING_MODEL)`

**Parameters**:
- `text` (str): Input text to embed
- `model` (str): Embedding model identifier

**Behavior**:
1. Checks if embedding exists in cache
2. If cached: Returns stored embedding immediately
3. If not cached:
   - Calls OpenAI API to generate embedding
   - Stores result in cache
   - Persists cache to disk
   - Returns embedding

**Returns**: 1536-dimensional embedding vector

**Cost Savings**: Dramatically reduces API expenses through intelligent caching

In [ ]:
def get_embedding(text, model=EMBEDDING_MODEL):
    if (text, model) not in embedding_cache:
        response = client.embeddings.create(
            model=model,
            input=text
        )
        embedding_cache[(text, model)] = response.data[0].embedding
        with open(CACHE_PATH, "wb") as f:
            pickle.dump(embedding_cache, f)
    return embedding_cache[(text, model)]

## ⚡ Step 6: Generate Embeddings for All Articles

**Purpose**: Create vector representations for all articles in the dataset

**Process**:
1. Extract all article descriptions from DataFrame
2. Iterate through each description with progress tracking
3. Generate embeddings using `get_embedding()` function
4. Accumulate embeddings in list
5. Convert to NumPy array for efficient computation

**Performance**:
- Uses `tqdm` for progress visualization
- Leverages caching to skip redundant API calls
- Output: 2D array of shape `(n_articles, 1536)`

**Result**: `embeddings` variable ready for similarity calculations

In [ ]:
descriptions = df["description"].tolist()
embeddings = []
for text in tqdm(descriptions):
    embeddings.append(get_embedding(text))
embeddings = np.array(embeddings)

## 🔍 Step 7: Recommendation Function (Programmatic Interface)

**Purpose**: Find semantically similar articles based on cosine similarity

**Function**: `recommend(index, k=5)`

**Parameters**:
- `index` (int): Index of the source article in the dataset
- `k` (int): Number of recommendations to return (default: 5)

**Algorithm**:
1. Extract source article embedding
2. Compute cosine similarity between source and all articles
3. Rank articles by similarity score (highest first)
4. Skip the source article itself
5. Display top-k recommendations with similarity scores

**Output Format**:
- Source article text
- Top-k recommendations with similarity scores (0-1 scale)
- Sorted by semantic relevance

**Use Case**: Programmatic access for batch recommendations or integration with other systems

In [ ]:
def recommend(index, k=5):
    query = embeddings[index].reshape(1, -1)
    scores = cosine_similarity(query, embeddings)[0]
    ranked = scores.argsort()[::-1]
    print("SOURCE ARTICLE:\n")
    print(descriptions[index])
    print("\n----------------------------\n")
    count = 0
    for i in ranked:
        if i == index:
            continue
        count += 1
        print(f"Recommendation #{count}")
        print(descriptions[i])
        print(f"Similarity: {scores[i]:.3f}\n")
        if count >= k:
            break

## 🎨 Step 8: Interactive Gradio Interface

**Purpose**: Provide user-friendly web interface for exploring recommendations

**Features**:
- **Input Controls**:
  - Article Index Slider: Select any article from dataset
  - Recommendations Slider: Choose 1-10 recommendations
- **Input Validation**: Prevents invalid indices and parameters
- **Rich Output**: Markdown-formatted results with emoji indicators
- **Professional UI**: Soft theme for better UX

**Functionality**:
- Real-time recommendation generation
- Similarity scores displayed for transparency
- Error handling and user feedback
- Shareable interface via public URL (with `share=True`)

**Launch Behavior**:
- Creates local Gradio app on localhost
- Generates optional public link for sharing
- Accessible from any web browser

**Ideal For**: Non-technical users, demonstrations, and production deployments

In [ ]:
import gradio as gr

def get_recommendations(article_index, num_recommendations):
    """Get recommendations based on article index"""
    try:
        article_index = int(article_index)
        num_recommendations = int(num_recommendations)
        
        if article_index < 0 or article_index >= len(descriptions):
            return "Error: Invalid article index. Please enter a value between 0 and " + str(len(descriptions) - 1)
        
        if num_recommendations < 1 or num_recommendations > 10:
            return "Error: Number of recommendations should be between 1 and 10"
        
        query = embeddings[article_index].reshape(1, -1)
        scores = cosine_similarity(query, embeddings)[0]
        ranked = scores.argsort()[::-1]
        
        result = "📰 **SOURCE ARTICLE:**\n\n"
        result += descriptions[article_index] + "\n\n"
        result += "---\n\n"
        result += "✨ **RECOMMENDATIONS:**\n\n"
        
        count = 0
        for i in ranked:
            if i == article_index:
                continue
            count += 1
            result += f"**#{count}** | Similarity: {scores[i]:.3f}\n\n"
            result += descriptions[i] + "\n\n---\n\n"
            if count >= num_recommendations:
                break
        
        return result
    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio interface
iface = gr.Interface(
    fn=get_recommendations,
    inputs=[
        gr.Slider(
            minimum=0,
            maximum=len(descriptions) - 1,
            step=1,
            label="Article Index",
            info=f"Select an article (0-{len(descriptions) - 1})"
        ),
        gr.Slider(
            minimum=1,
            maximum=10,
            step=1,
            value=5,
            label="Number of Recommendations",
            info="How many recommendations do you want?"
        )
    ],
    outputs=gr.Markdown(label="Results"),
    title="📚 Content-Based Recommendation System",
    description="Find similar articles using OpenAI embeddings and cosine similarity",
    theme=gr.themes.Soft(),
    allow_flagging="never"
)

# Launch the interface
iface.launch(share=True)